### **How trustworthy is your local representative?**

#### Data Sources:
politifact api documentation: http://static.politifact.com/api/v2apidoc.html <br>
govtrack api documentation: https://www.govtrack.us/developers/api <br>
propublica: https://projects.propublica.org/api-docs/congress-api/ <br>
congressional misconduct database: https://www.govtrack.us/misconduct <br>
news api: https://newsapi.org/docs <br>
votesmart: https://github.com/votesmart/python-votesmart/tree/master

### To do: April 15

1. Crawl truth-o-meter data ✅
2. Join truth-o-meter, misconduct db, and propublica **(in-progress)**
3. Structure bill vote
4. Incorporate news headlines
5. incorprate tweets (?)


In [1]:
import requests

import json

import pandas as pd
import numpy as np
import datetime as dt

from IPython.display import clear_output

import os
import time
import ast

import requests
from lxml import html

### Politifact
politifact api documentation: http://static.politifact.com/api/v2apidoc.html 

In [2]:
def get_politifact(limit,offset):
    pf_url = 'http://www.politifact.com/api/v/2/statement/'

    params = {
        'format' : 'json',
        'edition__edition_slug':'truth-o-meter',
        'limit' : limit,
        'offset' : offset,
        'order_by' : 'ruling_date',
    }

    resp = requests.get(pf_url,params=params).json()
    pf = pd.DataFrame(resp['objects'])
    return pf

csv_path = '/Users/Xie/Documents/School/Senior Spring/PPDS/Data/Politifact.csv'

def restart_politifact(file_path):
    
    """Restart Politifact Crawl"""
    
    df = get_politifact(100,0)
    df.to_csv(csv_path)
    
def refresh_politifact(file_path):
    """Refresh Politifact CSV"""
    cols = get_politifact(10,0).columns
    df = pd.read_csv(file_path)
    print('Pre-refresh length:',len(df))
    df = df.drop_duplicates('statement')[cols]
    print('Post-refreshed length:',len(df))

### get_politifact.py script (markdown)

Unix commands: <br>
pause: ctr + c <br>
resume: fg <br>
```python
import time
import pandas as pd
import requests
import json

print('\nGet Politifact\n')

limit = 100

print('*'*20)

def get_politifact(limit,offset):

    """Crawl Politifact API"""

    pf_url = 'http://www.politifact.com/api/v/2/statement/'

    params = {
        'format' : 'json',
        'edition__edition_slug':'truth-o-meter',
        'limit' : limit,
        'offset' : offset,
        'order_by' : 'ruling_date',
    }

    resp = requests.get(pf_url,params=params).json()
    pf = pd.DataFrame(resp['objects'])
    return pf

file_path = '/Users/Xie/Documents/School/Senior Spring/PPDS/Data/Politifact.json'

run = True
while run:
    try:
        t1 = time.time()
        old_df = pd.read_json(file_path)

        offset = len(old_df)+1

        print('data_length:',len(old_df))
        print('Offset:',offset)
        print('Getting data...')

        new_df = get_politifact(limit,offset)
        cols = new_df.columns

        df = old_df.append(new_df).drop_duplicates('statement').reset_index()[cols]
        df.to_json(file_path)

        print('---')
        print('data_length:',len(df))

        exp = offset+limit
        print('Expected offset:',exp)
        print('---')

        offset = len(df)+1
        print('New offset:',offset)

        timer = time.time() - t1
        print('runtime: {rt} seconds'.format(rt=round(timer,2)))

        if exp < offset:
            print('#'*10,'OFFSET ERROR','#'*10)
        elif offset > exp:
            print('OFFSET Diff:',offset-exp)
            
        time.sleep(15)

    except:
        time.sleep(30)

    print('='*20)
```

### Politifact Cleaning

In [3]:
print(os.getcwd())

/Users/Xie/Documents/School/Senior Spring/PPDS


In [4]:
json_path = '/Users/Xie/Documents/School/Senior Spring/PPDS/Data/Politifact.json'
pf = pd.read_json(json_path)
pf.sort_values('statement_date',ascending=False,inplace=True)
pf.reset_index(inplace=True)

pf['statement_date'] = pd.to_datetime(pf['statement_date'])
pf['ruling_date'] = pd.to_datetime(pf['ruling_date'])
pf['statement'] = (pf['statement'].str.replace('<p>',"").str.replace('</p>','')
                   .str.replace('"','').str.replace('&quot;','')
                   .str.replace('\r','').str.replace('\n','').str.replace('&#39;','\''))
pf['ruling_comments'] = (pf['ruling_comments'].str.replace('<p>',"").str.replace('</p>','')
                   .str.replace('"','').str.replace('&quot;','')
                   .str.replace('\r','').str.replace('\n','').str.replace('&#39;','\''))

pf['ruling_slug'] = [ruling['ruling_slug'] for ruling in pf['ruling']]

pf['name_slug'] = [speaker['name_slug'] for speaker in pf['speaker']]
pf['home_state'] = [speaker['home_state'] for speaker in pf['speaker']]
pf['party'] = [speaker['party']['party_slug'] for speaker in pf['speaker']]
pf['first_name'] = [speaker['first_name'] for speaker in pf['speaker']]
pf['last_name'] = [speaker['last_name'] for speaker in pf['speaker']]

pf['statement_type'] = [statement['statement_type'] for statement in pf['statement_type']]

pf['subject_slug'] = [[subject['subject_slug'] for subject in subjects] for subjects in pf['subject']]



In [5]:
sub = 'last_name'
main = 'speaker'
test_num = 10

import random

test = [random.randint(0,len(pf)) for i in range(test_num)]

for n in test:
    print(pf[sub].iloc[n] == pf.iloc[n][main][sub],'\t',pf[sub].iloc[n],pf.iloc[n][main][sub])

True 	 Obama Obama
True 	 Chain email Chain email
True 	 Perry Perry
True 	 Gardner Gardner
True 	 Boxer Boxer
True 	 Fehrnstrom Fehrnstrom
True 	 Gingrich Gingrich
True 	 Stupak Stupak
True 	 Gonzales Gonzales
True 	 Reich Reich


In [6]:
pf.columns

Index(['index', 'art', 'author', 'canonical_url', 'edition', 'editor',
       'facebook_headline', 'id', 'in_future', 'is_pundit', 'make_public',
       'preview', 'researcher', 'resource_uri', 'ruling', 'ruling_comments',
       'ruling_comments_date', 'ruling_date', 'ruling_headline',
       'ruling_headline_slug', 'ruling_link_text', 'ruling_state',
       'source_documents', 'sources', 'speaker', 'statement',
       'statement_context', 'statement_date', 'statement_type', 'subject',
       'target', 'twitter_headline', 'ruling_slug', 'name_slug', 'home_state',
       'party', 'first_name', 'last_name', 'subject_slug'],
      dtype='object')

In [7]:
politifact = pf[['ruling_slug','first_name','last_name','name_slug',
                 'home_state','party','statement',
                 'subject_slug','ruling_comments','ruling_date',
                 'statement_context','statement_type','statement_date','twitter_headline'
                 ,'sources']]
politifact

,ruling_slug,first_name,last_name,name_slug,home_state,party,statement,subject_slug,ruling_comments,ruling_date,statement_context,statement_type,statement_date,twitter_headline,sources
0,barely-true,Tina,Smith,tina-smith,Minnesota,democrat,"On average, American women only earn 80 cents ...","[diversity, jobs, women]",On April 10 -- Equal Pay Day -- Sen. Tina Smit...,2018-04-13 12:27:18,a tweet,Claim,2018-04-10,Do women get only 80 percent of the pay men do...,"<p>Tina Smith, <a href=""https://twitter.com/Se..."
1,mostly-true,Donald,Trump,donald-trump,New York,republican,When a car is sent to the United States from C...,"[china, trade]",President Donald Trump took to Twitter to bemo...,2018-04-09 17:06:15,in a tweet,Claim,2018-04-09,Does China slap a 25 percent tariff on America...,"<p><a href=""https://twitter.com/realDonaldTrum..."
2,barely-true,Donald,Trump,donald-trump,New York,republican,EPA administrator Scott Pruitt's short-term re...,[ethics],When Environmental Protection Agency administr...,2018-04-10 11:09:40,a tweet,Claim,2018-04-07,Did EPA's Pruitt pay below market rent?,"<p>Donald Trump, <a href=""https://twitter.com/..."
3,barely-true,Donald,Trump,donald-trump,New York,republican,Says Scott Pruitt&rsquo;s security spending wa...,"[environment, federal-budget, homeland-security]",President Donald Trump came out in defense of ...,2018-04-10 11:08:25,a tweet,Claim,2018-04-07,Comparing Scott Pruitt's security spending wit...,"<p>Donald Trump, <a href=""https://twitter.com/..."
4,barely-true,Donald,Trump,donald-trump,New York,republican,This will be the last time &mdash; April &mdas...,[taxes],If you are struggling to finish and file your ...,2018-04-09 08:00:00,an event in West Virginia,Claim,2018-04-05,Can taxpayers expect card-sized tax forms next...,"<p>Donald Trump, <a href=""https://www.whitehou..."
5,pants-fire,Donald,Trump,donald-trump,New York,republican,"In many places, like California, the same pers...",[elections],In a speech in West Virginia that was billed a...,2018-04-06 10:58:37,a speech in West Virginia,Claim,2018-04-05,No proof of Trump's claim that millions voted ...,"<p>Sharad Goel and Marc Meredith, et. al <a hr..."
6,pants-fire,,Bloggers,blog-posting,,none,YouTube witness makes shocking admission on li...,[fake-news],The day after a woman opened fire at the YouTu...,2018-04-06 12:51:31,a fake news blog post,Claim,2018-04-04,"Fake news says CNN ""pulled the plug"" during Yo...","<p>Daily Vine, <a href=""http://sharered.com/cn..."
7,barely-true,Todd,Rokita,todd-rokita,,republican,Luke Messer? He plotted with the Never-Trumper...,[candidates-biography],Three Republican candidates competing for a U....,2018-04-13 10:00:00,an ad,Claim,2018-04-03,Did Luke Messer plot to steal the nomination f...,"<p>Youtube, <a href=""https://www.youtube.com/w..."
8,barely-true,Donald,Trump,donald-trump,New York,republican,We&rsquo;ve started building the wall.,"[homeland-security, immigration]",President Donald Trump claimed his administrat...,2018-04-04 11:27:23,a press conference,Claim,2018-04-03,.@realdonaldtrump claims his promised border w...,"<p>Twitter, @CBSNews <a href=""https://twitter...."
9,mostly-true,Ruben,Gallego,ruben-gallego,Arizona,democrat,For what it would cost the (National) Guard to...,"[corrections-and-updates, homeland-security, h...",President Donald Trump&rsquo;s decision to sen...,2018-04-05 16:00:07,a tweet,Claim,2018-04-03,Is it cheaper to house a homeless vet than use...,"<p>Ruben Gallego, <a href=""https://twitter.com..."


### Govtrack (Deprecated)
govtrack api documentation: https://www.govtrack.us/developers/api 

In [8]:
# Data on legislators

person = 'https://www.govtrack.us/api/v2/person'
role = 'https://www.govtrack.us/api/v2/role'

params = {
    'current' : 'true'
}

resp = requests.get(person).json()

legis = pd.DataFrame(resp['objects'])
hillary = legis[legis['lastname']=='Clinton']['bioguideid'].iloc[0]

### Propublica
propublica: https://projects.propublica.org/api-docs/congress-api/ 

In [9]:
from congress import Congress
propublica_key = 'wAxQ7sF8gcXCBRnY3lzegT23aljM4saALOb6JPlR'
congress = Congress(propublica_key)

cong = 80 # Earliest congress is 80
cong = 115 # Latest is 115

st = congress.members.filter('senate',congress = cong)
hs = congress.members.filter('house', congress = cong)

senate_cols = ['id','lis_id','state','state_rank','party',
               'first_name','last_name','middle_name',
               'gender','date_of_birth','in_office',
               'votes_with_party_pct','missed_votes_pct','missed_votes',
               'next_election','title','seniority', # Seniority = years served
               'leadership_role','dw_nominate', # look into dw_nominate
               'twitter_account']

house_cols = ['id','state','district','party',
              'first_name','last_name','middle_name',
              'gender','date_of_birth','in_office',
              'votes_with_party_pct','missed_votes_pct','missed_votes',
              'next_election','title','seniority',
              'leadership_role','dw_nominate',
              'twitter_account']

senate = pd.DataFrame(st[0]['members'])[senate_cols]
house = pd.DataFrame(hs[0]['members'])[house_cols]

### Connecting Propublica to Politifact

In [34]:
# Connect Senate to politifact
senate['name_slug'] = senate['first_name'].str.lower() + '-' + senate['last_name'].str.lower()

filter1 = senate[~senate['name_slug'].isin(politifact['name_slug'])]
filter2 = filter1[filter1['last_name'].isin(politifact['last_name'])]
filter3 = filter1[~filter1['last_name'].isin(politifact['last_name'])]

new_slugs = list()

for i in range(len(filter2)):
    name = ['first_name','last_name']
    col = 'last_name'
    query = filter2['last_name'].iloc[i]
    result = politifact[politifact[col] == query]
    slug = result['name_slug'].iloc[0]
    
    new_slugs.append(slug)

    #print(filter2[name].iloc[i])
    #display(result[name])
    #print(slug)
    #print('='*30)

new_slugs = [slug if slug != 'joe-kennedy' else 'john-kennedy' for slug in new_slugs]
new_slugs = [slug if slug != 'mark-udall' else 'tom-udall' for slug in new_slugs]

In [11]:
filter2.loc[:,'name_slug'] = new_slugs

/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [12]:
# Change name slugs
for index in filter2.index:
    print('old:',senate.loc[index]['name_slug'] == filter2.loc[index]['name_slug'],
          senate.loc[index]['name_slug'],'\t', filter2.loc[index]['name_slug'])
    senate.loc[index,'name_slug'] = filter2.loc[index]['name_slug']
    
print('='*20)

for index in filter2.index:
    print('new:',senate.loc[index]['name_slug'] == filter2.loc[index]['name_slug'],
          senate.loc[index]['name_slug'],'\t', filter2.loc[index]['name_slug'])

old: False shelley-capito 	 shelley-moore-capito
old: False benjamin-cardin 	 ben-cardin
old: False christopher-coons 	 chris-coons
old: False michael-enzi 	 mike-enzi
old: False charles-grassley 	 chuck-grassley
old: True john-kennedy 	 john-kennedy
old: False christopher-murphy 	 chris-murphy
old: False jim-risch 	 james-risch
old: False bernard-sanders 	 bernie-s
old: False patrick-toomey 	 pat-toomey
old: True tom-udall 	 tom-udall
old: False chris-van hollen 	 chris-van-hollen
new: True shelley-moore-capito 	 shelley-moore-capito
new: True ben-cardin 	 ben-cardin
new: True chris-coons 	 chris-coons
new: True mike-enzi 	 mike-enzi
new: True chuck-grassley 	 chuck-grassley
new: True john-kennedy 	 john-kennedy
new: True chris-murphy 	 chris-murphy
new: True james-risch 	 james-risch
new: True bernie-s 	 bernie-s
new: True pat-toomey 	 pat-toomey
new: True tom-udall 	 tom-udall
new: True chris-van-hollen 	 chris-van-hollen


In [13]:
pd.DataFrame(st[0]['members']).columns

Index(['api_uri', 'contact_form', 'crp_id', 'cspan_id', 'date_of_birth',
       'dw_nominate', 'facebook_account', 'fax', 'fec_candidate_id',
       'first_name', 'gender', 'google_entity_id', 'govtrack_id', 'icpsr_id',
       'id', 'ideal_point', 'in_office', 'last_name', 'leadership_role',
       'lis_id', 'middle_name', 'missed_votes', 'missed_votes_pct',
       'next_election', 'ocd_id', 'office', 'party', 'phone', 'rss_url',
       'senate_class', 'seniority', 'short_title', 'state', 'state_rank',
       'suffix', 'title', 'total_present', 'total_votes', 'twitter_account',
       'url', 'votes_with_party_pct', 'votesmart_id', 'youtube_account'],
      dtype='object')

In [14]:
pd.DataFrame(hs[0]['members']).columns

Index(['api_uri', 'at_large', 'contact_form', 'crp_id', 'cspan_id',
       'date_of_birth', 'district', 'dw_nominate', 'facebook_account', 'fax',
       'fec_candidate_id', 'first_name', 'gender', 'geoid', 'google_entity_id',
       'govtrack_id', 'icpsr_id', 'id', 'ideal_point', 'in_office',
       'last_name', 'leadership_role', 'middle_name', 'missed_votes',
       'missed_votes_pct', 'next_election', 'ocd_id', 'office', 'party',
       'phone', 'rss_url', 'seniority', 'short_title', 'state', 'suffix',
       'title', 'total_present', 'total_votes', 'twitter_account', 'url',
       'votes_with_party_pct', 'votesmart_id', 'youtube_account'],
      dtype='object')

In [31]:
url = congress.votes.by_month('house')['votes'][0]['source']
response = requests.get(url)
print(response.text[:1000])

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE rollcall-vote PUBLIC "-//US Congress//DTDs/vote v1.0 20031119 //EN" "http://clerk.house.gov/evs/vote.dtd">
<?xml-stylesheet type="text/xsl" href="http://clerk.house.gov/evs/vote.xsl"?>
<rollcall-vote>
<vote-metadata>
<majority>R</majority>
<congress>115</congress>
<session>2nd</session>
<chamber>U.S. House of Representatives</chamber>
<rollcall-num>141</rollcall-num>
<legis-num>S 167</legis-num>
<vote-question>On Motion to Suspend the Rules and Pass</vote-question>
<vote-type>2/3 YEA-AND-NAY</vote-type>
<vote-result>Passed</vote-result>
<action-date>16-Apr-2018</action-date>
<action-time time-etz="19:03">7:03 PM</action-time>
<vote-desc>To designate a National Memorial to Fallen Educators at the National Teachers Hall of Fame in Emporia, Kansas</vote-desc>
<vote-totals>
<totals-by-party-header>
<party-header>Party</party-header>
<yea-header>Yeas</yea-header>
<nay-header>Nays</nay-header>
<present-header>Answered â


In [21]:
url = congress.votes.by_month('house')['votes'][0]['source']

response = requests.get(url)
doc = html.fromstring(response.content)

print(doc.xpath('//vote-desc')[0].text_content())
print(doc.xpath('//legis-num')[0].text_content())
print('\n'+'*'*20+'\n')

for record in doc.xpath('//vote-data/recorded-vote'):
    rep_id = record.xpath('./legislator')[0].get('name-id')
    party = record.xpath('./legislator')[0].get('party')
    state = record.xpath('./legislator')[0].get('state')
    name = record.xpath('./legislator')[0].text_content()
    vote = record.xpath('./vote')[0].text_content()
    
    print(name,party+'-'+state)
    print(rep_id)
    print(vote)
    print('-'*5)

To designate a National Memorial to Fallen Educators at the National Teachers Hall of Fame in Emporia, Kansas
S 167

********************

Abraham R-LA
A000374
Yea
-----
Adams D-NC
A000370
Yea
-----
Aderholt R-AL
A000055
Yea
-----
Aguilar D-CA
A000371
Yea
-----
Allen R-GA
A000372
Yea
-----
Amash R-MI
A000367
Nay
-----
Amodei R-NV
A000369
Yea
-----
Arrington R-TX
A000375
Yea
-----
Babin R-TX
B001291
Yea
-----
Bacon R-NE
B001298
Yea
-----
Banks (IN) R-IN
B001299
Yea
-----
Barletta R-PA
B001269
Yea
-----
Barr R-KY
B001282
Yea
-----
Barragán D-CA
B001300
Yea
-----
Barton R-TX
B000213
Yea
-----
Bass D-CA
B001270
Not Voting
-----
Beatty D-OH
B001281
Yea
-----
Bera D-CA
B001287
Yea
-----
Bergman R-MI
B001301
Yea
-----
Beyer D-VA
B001292
Yea
-----
Biggs R-AZ
B001302
Yea
-----
Bilirakis R-FL
B001257
Yea
-----
Bishop (GA) D-GA
B000490
Yea
-----
Bishop (MI) R-MI
B001293
Yea
-----
Bishop (UT) R-UT
B001250
Yea
-----
Black R-TN
B001273
Not Voting
-----
Blackburn R-TN
B001243
Yea
-----
Blum R-IA
B001

McHenry R-NC
M001156
Yea
-----
McKinley R-WV
M001180
Yea
-----
McMorris Rodgers R-WA
M001159
Yea
-----
McNerney D-CA
M001166
Yea
-----
McSally R-AZ
M001197
Not Voting
-----
Meadows R-NC
M001187
Yea
-----
Meehan R-PA
M001181
Yea
-----
Meeks D-NY
M001137
Yea
-----
Meng D-NY
M001188
Not Voting
-----
Messer R-IN
M001189
Yea
-----
Mitchell R-MI
M001201
Yea
-----
Moolenaar R-MI
M001194
Yea
-----
Mooney (WV) R-WV
M001195
Yea
-----
Moore D-WI
M001160
Yea
-----
Moulton D-MA
M001196
Yea
-----
Mullin R-OK
M001190
Yea
-----
Murphy (FL) D-FL
M001202
Yea
-----
Nadler D-NY
N000002
Yea
-----
Napolitano D-CA
N000179
Yea
-----
Neal D-MA
N000015
Not Voting
-----
Newhouse R-WA
N000189
Yea
-----
Noem R-SD
N000184
Not Voting
-----
Nolan D-MN
N000127
Yea
-----
Norcross D-NJ
N000188
Yea
-----
Norman R-SC
N000190
Yea
-----
Nunes R-CA
N000181
Yea
-----
O'Halleran D-AZ
O000171
Yea
-----
O'Rourke D-TX
O000170
Yea
-----
Olson R-TX
O000168
Not Voting
-----
Palazzo R-MS
P000601
Yea
-----
Pallone D-NJ
P000034
Yea
---

In [22]:
print(congress.votes.by_month('senate')['votes'][1]['url'])

https://www.senate.gov/legislative/LIS/roll_call_lists/roll_call_vote_cfm.cfm?congress=115&session=2&vote=00073


In [23]:
url = congress.votes.by_month('senate')['votes'][1]['source']

response = requests.get(url)
doc = html.fromstring(response.content)

print(doc.xpath('//vote_title')[0].text_content())
print('\n'+'*'*20+'\n')

for member in doc.xpath('//member'):
    print(member.xpath('./member_full')[0].text_content())
    print(member.xpath('./first_name')[0].text_content())
    print(member.xpath('./vote_cast')[0].text_content())
    print(member.xpath('./lis_member_id')[0].text_content())
    print('-'*10)

Motion to Invoke Cloture Re: Rebecca Grady Jennings, of Kentucky, to be U.S. District Judge for the Western District of Kentucky

********************

Alexander (R-TN)
Lamar
Yea
S289
----------
Baldwin (D-WI)
Tammy
Yea
S354
----------
Barrasso (R-WY)
John
Yea
S317
----------
Bennet (D-CO)
Michael
Yea
S330
----------
Blumenthal (D-CT)
Richard
Yea
S341
----------
Blunt (R-MO)
Roy
Yea
S342
----------
Booker (D-NJ)
Cory
Yea
S370
----------
Boozman (R-AR)
John
Yea
S343
----------
Brown (D-OH)
Sherrod 
Yea
S307
----------
Burr (R-NC)
Richard
Yea
S300
----------
Cantwell (D-WA)
Maria
Yea
S275
----------
Capito (R-WV)
Shelley
Yea
S372
----------
Cardin (D-MD)
Ben
Yea
S308
----------
Carper (D-DE)
Thomas
Yea
S277
----------
Casey (D-PA)
Bob
Yea
S309
----------
Cassidy (R-LA)
Bill
Yea
S373
----------
Collins (R-ME)
Susan
Yea
S252
----------
Coons (D-DE)
Christopher
Not Voting
S337
----------
Corker (R-TN)
Bob
Yea
S310
----------
Cornyn (R-TX)
John
Yea
S287
----------
Cortez Masto (D-NV)
Catheri

In [24]:
# compare
congress.members.compare('S000033','C001098','senate',congress=cong)

{'agree_percent': 16.4,
 'chamber': 'Senate',
 'common_votes': 378,
 'congress': '115',
 'disagree_percent': 83.6,
 'disagree_votes': 316,
 'first_member_api_uri': 'https://api.propublica.org/congress/v1/members/S000033.json',
 'first_member_id': 'S000033',
 'second_member_api_uri': 'https://api.propublica.org/congress/v1/members/C001098.json',
 'second_member_id': 'C001098'}

In [25]:
#bills
pd.DataFrame(congress.bills.by_member(senate['id'][0])['bills']).head(3)

,active,bill_id,bill_type,bill_uri,committees,congress,congressdotgov_url,cosponsors,cosponsors_by_party,enacted,...,sponsor_id,sponsor_name,sponsor_party,sponsor_state,sponsor_title,sponsor_uri,summary,summary_short,title,vetoed
0,False,s2513-115,s,https://api.propublica.org/congress/v1/115/bil...,"Senate Health, Education, Labor, and Pensions ...",115,https://www.congress.gov/bill/115th-congress/s...,15,{'R': 15},None,...,A000360,Lamar Alexander,R,TN,Sen.,https://api.propublica.org/congress/v1/members...,,,A bill to improve school safety and mental hea...,None
1,False,s2509-115,s,https://api.propublica.org/congress/v1/115/bil...,Senate Energy and Natural Resources Committee,115,https://www.congress.gov/bill/115th-congress/s...,7,"{'R': 4, 'D': 2, 'I': 1}",None,...,A000360,Lamar Alexander,R,TN,Sen.,https://api.propublica.org/congress/v1/members...,,,A bill to establish the National Park Restorat...,None
2,True,s2434-115,s,https://api.propublica.org/congress/v1/115/bil...,"Senate Health, Education, Labor, and Pensions ...",115,https://www.congress.gov/bill/115th-congress/s...,0,{},None,...,A000360,Lamar Alexander,R,TN,Sen.,https://api.propublica.org/congress/v1/members...,,,"A bill to amend the Federal Food, Drug, and Co...",None


In [26]:
# same thing, above is probably better
pd.DataFrame(congress.members.bills(senate['id'][0])['bills']).head(3)

,active,bill_id,bill_type,bill_uri,committees,congress,congressdotgov_url,cosponsors,cosponsors_by_party,enacted,...,sponsor_id,sponsor_name,sponsor_party,sponsor_state,sponsor_title,sponsor_uri,summary,summary_short,title,vetoed
0,False,s2513-115,s,https://api.propublica.org/congress/v1/115/bil...,"Senate Health, Education, Labor, and Pensions ...",115,https://www.congress.gov/bill/115th-congress/s...,15,{'R': 15},None,...,A000360,Lamar Alexander,R,TN,Sen.,https://api.propublica.org/congress/v1/members...,,,A bill to improve school safety and mental hea...,None
1,False,s2509-115,s,https://api.propublica.org/congress/v1/115/bil...,Senate Energy and Natural Resources Committee,115,https://www.congress.gov/bill/115th-congress/s...,7,"{'R': 4, 'D': 2, 'I': 1}",None,...,A000360,Lamar Alexander,R,TN,Sen.,https://api.propublica.org/congress/v1/members...,,,A bill to establish the National Park Restorat...,None
2,True,s2434-115,s,https://api.propublica.org/congress/v1/115/bil...,"Senate Health, Education, Labor, and Pensions ...",115,https://www.congress.gov/bill/115th-congress/s...,0,{},None,...,A000360,Lamar Alexander,R,TN,Sen.,https://api.propublica.org/congress/v1/members...,,,"A bill to amend the Federal Food, Drug, and Co...",None


In [27]:
print(dir(congress.bills))

['BASE_URI', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'amendments', 'apikey', 'by_member', 'cosponsors', 'fetch', 'get', 'http', 'introduced', 'major', 'passed', 'recent', 'related', 'subjects', 'upcoming', 'updated']


In [35]:
pd.DataFrame(congress.bills.passed('senate')['bills']).head()

,active,bill_id,bill_slug,bill_type,bill_uri,committee_codes,committees,congressdotgov_url,cosponsors,cosponsors_by_party,...,sponsor_name,sponsor_party,sponsor_state,sponsor_title,sponsor_uri,subcommittee_codes,summary,summary_short,title,vetoed
0,True,hr4547-115,hr4547,hr,https://api.propublica.org/congress/v1/115/bil...,"[SSFI, HSWM]",Senate Finance Committee,https://www.congress.gov/bill/115th-congress/h...,75,"{'D': 41, 'R': 34}",...,Sam Johnson,R,TX,Rep.,https://api.propublica.org/congress/v1/members...,[],Strengthening Protections for Social Security ...,Strengthening Protections for Social Security ...,"To amend titles II, VIII, and XVI of the Socia...",None
1,True,s772-115,s772,s,https://api.propublica.org/congress/v1/115/bil...,"[HSJU, SLIA]",House Judiciary Committee,https://www.congress.gov/bill/115th-congress/s...,6,"{'D': 4, 'R': 2}",...,John McCain,R,AZ,Sen.,https://api.propublica.org/congress/v1/members...,[],Ashlynne Mike AMBER Alert in Indian Country Ac...,Ashlynne Mike AMBER Alert in Indian Country Ac...,A bill to amend the PROTECT Act to make Indian...,None
2,True,s140-115,s140,s,https://api.propublica.org/congress/v1/115/bil...,"[SLIA, HSII]",House Natural Resources Committee,https://www.congress.gov/bill/115th-congress/s...,1,{'R': 1},...,Jeff Flake,R,AZ,Sen.,https://api.propublica.org/congress/v1/members...,[HSII13],(Sec. 1) This bill amends the White Mountain A...,(Sec. 1) This bill amends the White Mountain A...,A bill to amend the White Mountain Apache Trib...,None
3,True,hr3979-115,hr3979,hr,https://api.propublica.org/congress/v1/115/bil...,[HSII],House Natural Resources Committee,https://www.congress.gov/bill/115th-congress/h...,17,"{'R': 12, 'D': 5}",...,Hakeem Jeffries,D,NY,Rep.,https://api.propublica.org/congress/v1/members...,[HSII10],(This measure has not been amended since it wa...,(This measure has not been amended since it wa...,To amend the Fish and Wildlife Act of 1956 to ...,None
4,True,hr3445-115,hr3445,hr,https://api.propublica.org/congress/v1/115/bil...,[HSFA],House Foreign Affairs Committee,https://www.congress.gov/bill/115th-congress/h...,8,"{'D': 6, 'R': 2}",...,Edward Royce,R,CA,Rep.,https://api.propublica.org/congress/v1/members...,[],African Growth and Opportunity Act and Millenn...,African Growth and Opportunity Act and Millenn...,To enhance the transparency and accelerate the...,None


In [29]:
pd.DataFrame(congress.bills.subjects('s2513',congress=cong))['subjects'].head()

0    {'name': 'Elementary and secondary education',...
1    {'name': 'Congressional oversight', 'url_name'...
2    {'name': 'Crime prevention', 'url_name': 'crim...
3    {'name': 'Child health', 'url_name': 'child-he...
4    {'name': 'Building construction', 'url_name': ...
Name: subjects, dtype: object

In [30]:
pd.DataFrame(congress.members.bills(hillary)['bills']).head()

,active,bill_id,bill_type,bill_uri,committees,congress,congressdotgov_url,cosponsors,cosponsors_by_party,enacted,...,sponsor_id,sponsor_name,sponsor_party,sponsor_state,sponsor_title,sponsor_uri,summary,summary_short,title,vetoed
0,None,s211-111,s,https://api.propublica.org/congress/v1/111/bil...,Senate Finance,111,https://www.congress.gov/bill/111th-congress/s...,61,"{'R': 16, 'D': 43, 'ID': 1, 'I': 1}",None,...,C001041,Hillary Rodham Clinton,D,NY,,https://api.propublica.org/congress/v1/members...,,,Calling for 2-1-1 Act of 2009,None
1,None,s182-111,s,https://api.propublica.org/congress/v1/111/bil...,Senate Finance,111,https://www.congress.gov/bill/111th-congress/s...,42,"{'D': 41, 'I': 1}",None,...,C001041,Hillary Rodham Clinton,D,NY,,https://api.propublica.org/congress/v1/members...,,,Paycheck Fairness Act,None
2,None,sres727-110,sres,https://api.propublica.org/congress/v1/110/bil...,,110,https://www.congress.gov/bill/110th-congress/s...,3,{'D': 3},None,...,C001041,Hillary Rodham Clinton,D,NY,,https://api.propublica.org/congress/v1/members...,,,A resolution honoring the victims of the bombi...,None
3,None,sres720-110,sres,https://api.propublica.org/congress/v1/110/bil...,"Senate Health, Education, Labor, and Pensions",110,https://www.congress.gov/bill/110th-congress/s...,2,"{'D': 1, 'R': 1}",None,...,C001041,Hillary Rodham Clinton,D,NY,,https://api.propublica.org/congress/v1/members...,,,A resolution supporting the goals and ideals o...,None
4,None,s19-110,s,https://api.propublica.org/congress/v1/110/bil...,Senate Finance,110,https://www.congress.gov/bill/110th-congress/s...,0,{},None,...,C001041,Hillary Rodham Clinton,D,NY,,https://api.propublica.org/congress/v1/members...,,,A bill to amend the Internal Revenue Code of 1...,None
